In [1]:
# Doing all major library imports
import matplotlib.pyplot as plt
import scikitplot as skplt
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as stats
import re

from sklearn import datasets, metrics
from sklearn.linear_model import LinearRegression, LogisticRegression,LogisticRegressionCV 
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, KFold
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import Ridge, Lasso, ElasticNet, LinearRegression, RidgeCV, LassoCV, ElasticNetCV
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import roc_curve, auc, precision_recall_curve, average_precision_score
from matplotlib.colors import ListedColormap
from sklearn.pipeline import Pipeline, make_pipeline

plt.style.use('fivethirtyeight')

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import scikitplot as skplt
from matplotlib.colors import ListedColormap
from sklearn.metrics import classification_report, confusion_matrix

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.options.display.float_format = '{:.2f}'.format

import wbdata as wb

In [2]:
cpi = pd.read_csv ('corruption_edited.csv')
cpi.head()

,Country,country_code,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006,2005,2004,2003,2002,2001,2000,1999,1998,1997,1996,1995
0,Denmark,DNK,88,88,90.00,91.00,92.00,91.00,90.00,93.92,93,93,93,94,95,95,95,95,95,95,98,100,100,99,93.30,93.20
1,New Zealand,NZL,87,89,90.00,91.00,91.00,91.00,90.00,94.63,93,94,93,94,96,96,96,95,95,94,94,94,94,92,94.30,95.50
2,Finland,FIN,85,85,89.00,90.00,89.00,89.00,90.00,94.04,92,89,90,94,96,96,97,97,97,99,100,98,96,95,90.50,91.20
3,Singapore,SGP,85,84,84.00,85.00,84.00,86.00,87.00,91.67,93,92,92,93,94,94,93,94,93,92,91,91,91,87,88.00,92.60
4,Sweden,SWE,85,84,88.00,89.00,87.00,89.00,88.00,92.98,92,92,93,93,92,92,92,93,93,90,94,94,95,95,90.80,88.70


In [3]:
cc = {
'Aruba':'ABW',
'Afghanistan':'AFG',
'Africa':'AFR',
'Angola':'AGO',
'Albania':'ALB',
'Andorra':'AND',
'Andean Region':'ANR',
'Arab World':'ARB',
'United Arab Emirates':'ARE',
'Argentina':'ARG',
'Armenia':'ARM',
'American Samoa':'ASM',
'Antigua and Barbuda':'ATG',
'Australia':'AUS',
'Austria':'AUT',
'Azerbaijan':'AZE',
'Burundi':'BDI',
'East Asia & Pacific (IBRD-only countries)':'BEA',
'Europe & Central Asia (IBRD-only countries)':'BEC',
'Belgium':'BEL',
'Benin':'BEN',
'Burkina Faso':'BFA',
'Bangladesh':'BGD',
'Bulgaria':'BGR',
'IBRD countries classified as high income':'BHI',
'Bahrain':'BHR',
'Bahamas, The':'BHS',
'Bosnia and Herzegovina':'BIH',
'Latin America & the Caribbean (IBRD-only countries)':'BLA',
'Belarus':'BLR',
'Belize':'BLZ',
'Middle East & North Africa (IBRD-only countries)':'BMN',
'Bermuda':'BMU',
'Bolivia':'BOL',
'Brazil':'BRA',
'Barbados':'BRB',
'Brunei Darussalam':'BRN',
'Sub-Saharan Africa (IBRD-only countries)':'BSS',
'Bhutan':'BTN',
'Botswana':'BWA',
'Sub-Saharan Africa (IFC classification)':'CAA',
'Central African Republic':'CAF',
'Canada':'CAN',
'East Asia and the Pacific (IFC classification)':'CEA',
'Central Europe and the Baltics':'CEB',
'Europe and Central Asia (IFC classification)':'CEU',
'Switzerland':'CHE',
'Channel Islands':'CHI',
'Chile':'CHL',
'China':'CHN',
'Cote d\'Ivoire':'CIV',
'Latin America and the Caribbean (IFC classification)':'CLA',
'Middle East and North Africa (IFC classification)':'CME',
'Cameroon':'CMR',
'Congo, Dem. Rep.':'COD',
'Congo, Rep.':'COG',
'Colombia':'COL',
'Comoros':'COM',
'Cabo Verde':'CPV',
'Costa Rica':'CRI',
'South Asia (IFC classification)':'CSA',
'Caribbean small states':'CSS',
'Cuba':'CUB',
'Curacao':'CUW',
'Cayman Islands':'CYM',
'Cyprus':'CYP',
'Czech Republic':'CZE',
'East Asia & Pacific (IDA-eligible countries)':'DEA',
'Europe & Central Asia (IDA-eligible countries)':'DEC',
'Germany':'DEU',
'IDA countries classified as Fragile Situations':'DFS',
'Djibouti':'DJI',
'Latin America & the Caribbean (IDA-eligible countries)':'DLA',
'Dominica':'DMA',
'Middle East & North Africa (IDA-eligible countries)':'DMN',
'IDA countries not classified as Fragile Situations':'DNF',
'Denmark':'DNK',
'IDA countries in Sub-Saharan Africa not classified as fragile situations ':'DNS',
'Dominican Republic':'DOM',
'South Asia (IDA-eligible countries)':'DSA',
'IDA countries in Sub-Saharan Africa classified as fragile situations ':'DSF',
'Sub-Saharan Africa (IDA-eligible countries)':'DSS',
'IDA total, excluding Sub-Saharan Africa':'DXS',
'Algeria':'DZA',
'East Asia & Pacific (excluding high income)':'EAP',
'Early-demographic dividend':'EAR',
'East Asia & Pacific':'EAS',
'Europe & Central Asia (excluding high income)':'ECA',
'Europe & Central Asia':'ECS',
'Ecuador':'ECU',
'Egypt, Arab Rep.':'EGY',
'Euro area':'EMU',
'Eritrea':'ERI',
'Spain':'ESP',
'Estonia':'EST',
'Ethiopia':'ETH',
'European Union':'EUU',
'Fragile and conflict affected situations':'FCS',
'Finland':'FIN',
'Fiji':'FJI',
'France':'FRA',
'Faroe Islands':'FRO',
'Micronesia, Fed. Sts.':'FSM',
'IDA countries classified as fragile situations, excluding Sub-Saharan Africa':'FXS',
'Gabon':'GAB',
'United Kingdom':'GBR',
'Georgia':'GEO',
'Ghana':'GHA',
'Gibraltar':'GIB',
'Guinea':'GIN',
'Gambia, The':'GMB',
'Guinea-Bissau':'GNB',
'Equatorial Guinea':'GNQ',
'Greece':'GRC',
'Grenada':'GRD',
'Greenland':'GRL',
'Guatemala':'GTM',
'Guam':'GUM',
'Guyana':'GUY',
'High income':'HIC',
'Hong Kong SAR, China':'HKG',
'Honduras':'HND',
'Heavily indebted poor countries (HIPC)':'HPC',
'Croatia':'HRV',
'Haiti':'HTI',
'Hungary':'HUN',
'IBRD, including blend':'IBB',
'IBRD only':'IBD',
'IDA & IBRD total':'IBT',
'IDA total':'IDA',
'IDA blend':'IDB',
'Indonesia':'IDN',
'IDA only':'IDX',
'Isle of Man':'IMN',
'India':'IND',
'Not classified':'INX',
'Ireland':'IRL',
'Iran, Islamic Rep.':'IRN',
'Iraq':'IRQ',
'Iceland':'ISL',
'Israel':'ISR',
'Italy':'ITA',
'Jamaica':'JAM',
'Jordan':'JOR',
'Japan':'JPN',
'Kazakhstan':'KAZ',
'Kenya':'KEN',
'Kyrgyz Republic':'KGZ',
'Cambodia':'KHM',
'Kiribati':'KIR',
'St. Kitts and Nevis':'KNA',
'Korea, Rep.':'KOR',
'Kuwait':'KWT',
'Latin America & Caribbean (excluding high income)':'LAC',
'Lao PDR':'LAO',
'Lebanon':'LBN',
'Liberia':'LBR',
'Libya':'LBY',
'St. Lucia':'LCA',
'Latin America & Caribbean ':'LCN',
'Latin America and the Caribbean':'LCR',
'Least developed countries: UN classification':'LDC',
'Low income':'LIC',
'Liechtenstein':'LIE',
'Sri Lanka':'LKA',
'Lower middle income':'LMC',
'Low & middle income':'LMY',
'Lesotho':'LSO',
'Late-demographic dividend':'LTE',
'Lithuania':'LTU',
'Luxembourg':'LUX',
'Latvia':'LVA',
'Macao SAR, China':'MAC',
'St. Martin (French part)':'MAF',
'Morocco':'MAR',
'Central America':'MCA',
'Monaco':'MCO',
'Moldova':'MDA',
'Middle East (developing only)':'MDE',
'Madagascar':'MDG',
'Maldives':'MDV',
'Middle East & North Africa':'MEA',
'Mexico':'MEX',
'Marshall Islands':'MHL',
'Middle income':'MIC',
'North Macedonia':'MKD',
'Mali':'MLI',
'Malta':'MLT',
'Myanmar':'MMR',
'Middle East & North Africa (excluding high income)':'MNA',
'Montenegro':'MNE',
'Mongolia':'MNG',
'Northern Mariana Islands':'MNP',
'Mozambique':'MOZ',
'Mauritania':'MRT',
'Mauritius':'MUS',
'Malawi':'MWI',
'Malaysia':'MYS',
'North America':'NAC',
'North Africa':'NAF',
'Namibia':'NAM',
'New Caledonia':'NCL',
'Niger':'NER',
'Nigeria':'NGA',
'Nicaragua':'NIC',
'Netherlands':'NLD',
'Non-resource rich Sub-Saharan Africa countries, of which landlocked':'NLS',
'Norway':'NOR',
'Nepal':'NPL',
'Non-resource rich Sub-Saharan Africa countries':'NRS',
'Nauru':'NRU',
'IDA countries not classified as fragile situations, excluding Sub-Saharan Africa':'NXS',
'New Zealand':'NZL',
'OECD members':'OED',
'Oman':'OMN',
'Other small states':'OSS',
'Pakistan':'PAK',
'Panama':'PAN',
'Peru':'PER',
'Philippines':'PHL',
'Palau':'PLW',
'Papua New Guinea':'PNG',
'Poland':'POL',
'Pre-demographic dividend':'PRE',
'Puerto Rico':'PRI',
'Korea, Dem. People???s Rep.':'PRK',
'Portugal':'PRT',
'Paraguay':'PRY',
'West Bank and Gaza':'PSE',
'Pacific island small states':'PSS',
'Post-demographic dividend':'PST',
'French Polynesia':'PYF',
'Qatar':'QAT',
'Romania':'ROU',
'Resource rich Sub-Saharan Africa countries':'RRS',
'Resource rich Sub-Saharan Africa countries, of which oil exporters':'RSO',
'Russian Federation':'RUS',
'Rwanda':'RWA',
'South Asia':'SAS',
'Saudi Arabia':'SAU',
'Southern Cone':'SCE',
'Sudan':'SDN',
'Senegal':'SEN',
'Singapore':'SGP',
'Solomon Islands':'SLB',
'Sierra Leone':'SLE',
'El Salvador':'SLV',
'San Marino':'SMR',
'Somalia':'SOM',
'Serbia':'SRB',
'Sub-Saharan Africa (excluding high income)':'SSA',
'South Sudan':'SSD',
'Sub-Saharan Africa ':'SSF',
'Small states':'SST',
'Sao Tome and Principe':'STP',
'Suriname':'SUR',
'Slovak Republic':'SVK',
'Slovenia':'SVN',
'Sweden':'SWE',
'Eswatini':'SWZ',
'Sint Maarten (Dutch part)':'SXM',
'Sub-Saharan Africa excluding South Africa':'SXZ',
'Seychelles':'SYC',
'Syrian Arab Republic':'SYR',
'Turks and Caicos Islands':'TCA',
'Chad':'TCD',
'East Asia & Pacific (IDA & IBRD countries)':'TEA',
'Europe & Central Asia (IDA & IBRD countries)':'TEC',
'Togo':'TGO',
'Thailand':'THA',
'Tajikistan':'TJK',
'Turkmenistan':'TKM',
'Latin America & the Caribbean (IDA & IBRD countries)':'TLA',
'Timor-Leste':'TLS',
'Middle East & North Africa (IDA & IBRD countries)':'TMN',
'Tonga':'TON',
'South Asia (IDA & IBRD)':'TSA',
'Sub-Saharan Africa (IDA & IBRD countries)':'TSS',
'Trinidad and Tobago':'TTO',
'Tunisia':'TUN',
'Turkey':'TUR',
'Tuvalu':'TUV',
'Taiwan, China':'TWN',
'Tanzania':'TZA',
'Uganda':'UGA',
'Ukraine':'UKR',
'Upper middle income':'UMC',
'Uruguay':'URY',
'United States':'USA',
'Uzbekistan':'UZB',
'St. Vincent and the Grenadines':'VCT',
'Venezuela, RB':'VEN',
'British Virgin Islands':'VGB',
'Virgin Islands (U.S.)':'VIR',
'Vietnam':'VNM',
'Vanuatu':'VUT',
'World':'WLD',
'Samoa':'WSM',
'Kosovo':'XKX',
'Sub-Saharan Africa excluding South Africa and Nigeria':'XZN',
'Yemen, Rep.':'YEM',
'South Africa':'ZAF',
'Zambia':'ZMB',
'Zimbabwe':'ZWE',
    
    'Afghanistan, Islamic Republic of': 'AFG', 
    #'Anguilla',
    'Armenia, Republic of': 'ARM', 
    'Azerbaijan, Republic of': 'AZE',
    'Bahrain, Kingdom of': 'BHR', 
    'China, P.R.: Hong Kong' : 'HKG',
    'China, P.R.: Macao': 'MAC', 
    'China, P.R.: Mainland': 'CHN',
    'Congo, Democratic Republic of' : 'COD', 
    'Congo, Republic of': 'COG',
    "Côte d'Ivoire": 'CIV', 
    'Egypt': 'EGY', 
    #'Eswatini, Kingdom of',
    'Iran, Islamic Republic of':'IRN', 
    'Korea, Republic of': 'KOR',
    #'Kosovo, Republic of', 
    "Lao People's Democratic Republic" : 'LAO',
    #'Marshall Islands, Republic of', 
    #'Micronesia, Federated States of',
    #'Montserrat', 
    'North Macedonia, Republic of':'MKD',
    'Serbia, Republic of': 'SRB', 
    'São Tomé and Príncipe': 'STP',
    'Timor-Leste, Dem. Rep. of': 'TLS', 
    'Yemen, Republic of': 'YEM',
    
    
    'Bosnia & Herzegovina':'BIH', 
    'Gambia':'GMB', 
    'Iran':'IRN', 
    #'North Korea': ,
    'Trinidad & Tobago' : 'TTO', 
    'Venezuela' : 'VEN', 
    'Viet Nam': 'VNM', 
    'Yemen': 'YEM'
}

In [4]:
cpi['cc2'] = cpi.Country.replace (cc)
cpi.head()

,Country,country_code,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006,2005,2004,2003,2002,2001,2000,1999,1998,1997,1996,1995,cc2
0,Denmark,DNK,88,88,90.00,91.00,92.00,91.00,90.00,93.92,93,93,93,94,95,95,95,95,95,95,98,100,100,99,93.30,93.20,DNK
1,New Zealand,NZL,87,89,90.00,91.00,91.00,91.00,90.00,94.63,93,94,93,94,96,96,96,95,95,94,94,94,94,92,94.30,95.50,NZL
2,Finland,FIN,85,85,89.00,90.00,89.00,89.00,90.00,94.04,92,89,90,94,96,96,97,97,97,99,100,98,96,95,90.50,91.20,FIN
3,Singapore,SGP,85,84,84.00,85.00,84.00,86.00,87.00,91.67,93,92,92,93,94,94,93,94,93,92,91,91,91,87,88.00,92.60,SGP
4,Sweden,SWE,85,84,88.00,89.00,87.00,89.00,88.00,92.98,92,92,93,93,92,92,92,93,93,90,94,94,95,95,90.80,88.70,SWE


In [6]:
cpi[cpi.country_code != cpi.cc2]

,Country,country_code,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006,2005,2004,2003,2002,2001,2000,1999,1998,1997,1996,1995,cc2
14,Hong Kong,HKG,76,77,77.00,75.00,74.00,75.00,77.00,83.90,84,82,81,83,83,83,80,80,82,79,77,77,78,73,70.10,71.20,Hong Kong
21,United States of America,USA,71,75,74.00,76.00,74.00,73.00,73.00,71.36,71,75,73,72,73,76,75,75,77,76,78,75,75,76,76.60,77.90,United States of America
28,Bahamas,BHS,65,65,66.00,nan,71.00,71.00,71.00,72.89,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.00,0.00,Bahamas
31,Taiwan,TWN,63,63,61.00,62.00,61.00,61.00,61.00,61.36,58,56,57,57,59,59,56,57,56,59,55,56,53,50,49.80,50.80,Taiwan
42,Saint Vincent and the Grenadines,VCT,58,58,60.00,nan,62.00,62.00,62.00,57.61,0,64,65,61,0,0,0,0,0,0,0,0,0,0,0.00,0.00,Saint Vincent and the Grenadines
46,"Korea, South",KOR,57,54,53.00,54.00,55.00,55.00,56.00,53.56,54,55,56,51,51,50,45,43,45,42,40,38,42,43,50.20,42.90,"Korea, South"
49,Saint Lucia,LCA,55,55,60.00,nan,71.00,71.00,71.00,70.18,0,70,71,68,0,0,0,0,0,0,0,0,0,0,0.00,0.00,Saint Lucia
56,Slovakia,SVK,50,50,51.00,51.00,50.00,47.00,46.00,39.72,43,45,50,49,47,43,40,37,37,37,35,37,39,0,0.00,0.00,Slovakia
91,Swaziland,SWZ,38,39,nan,nan,43.00,39.00,37.00,30.82,32,36,36,33,25,27,0,0,0,0,0,0,0,0,0.00,0.00,Swaziland
94,Kosovo,KSV,37,39,36.00,33.00,33.00,33.00,34.00,28.51,28,0,0,0,0,0,0,0,0,0,0,0,0,0,0.00,0.00,XKX


In [7]:
cpi.pop ('cc2')

0                                   DNK
1                                   NZL
2                                   FIN
3                                   SGP
4                                   SWE
5                                   CHE
6                                   NOR
7                                   NLD
8                                   CAN
9                                   LUX
10                                  DEU
11                                  GBR
12                                  AUS
13                                  AUT
14                            Hong Kong
15                                  ISL
16                                  BEL
17                                  EST
18                                  IRL
19                                  JPN
20                                  FRA
21             United States of America
22                                  ARE
23                                  URY
24                                  BRB


In [8]:
cpi.head()

,Country,country_code,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006,2005,2004,2003,2002,2001,2000,1999,1998,1997,1996,1995
0,Denmark,DNK,88,88,90.00,91.00,92.00,91.00,90.00,93.92,93,93,93,94,95,95,95,95,95,95,98,100,100,99,93.30,93.20
1,New Zealand,NZL,87,89,90.00,91.00,91.00,91.00,90.00,94.63,93,94,93,94,96,96,96,95,95,94,94,94,94,92,94.30,95.50
2,Finland,FIN,85,85,89.00,90.00,89.00,89.00,90.00,94.04,92,89,90,94,96,96,97,97,97,99,100,98,96,95,90.50,91.20
3,Singapore,SGP,85,84,84.00,85.00,84.00,86.00,87.00,91.67,93,92,92,93,94,94,93,94,93,92,91,91,91,87,88.00,92.60
4,Sweden,SWE,85,84,88.00,89.00,87.00,89.00,88.00,92.98,92,92,93,93,92,92,92,93,93,90,94,94,95,95,90.80,88.70


In [9]:
cpi_melt = pd.melt(cpi, id_vars=['Country', 'country_code'])
cpi_melt.head()

,Country,country_code,variable,value
0,Denmark,DNK,2018,88.00
1,New Zealand,NZL,2018,87.00
2,Finland,FIN,2018,85.00
3,Singapore,SGP,2018,85.00
4,Sweden,SWE,2018,85.00


In [13]:
cpi_melt.columns = ['Country','country_code','date','Corruption Perception Index']
cpi_melt.head()

,Country,country_code,date,Corruption Perception Index
0,Denmark,DNK,2018,88.00
1,New Zealand,NZL,2018,87.00
2,Finland,FIN,2018,85.00
3,Singapore,SGP,2018,85.00
4,Sweden,SWE,2018,85.00


In [14]:
cpi_melt.to_csv ('corruption_index_output.csv', index=False)